# 1. Import Libraries:

In [109]:
import mediapipe as mp
import numpy as np
import cv2
import os

In [110]:
text_background = (198, 63, 88)  # PURPLE
corner_color = (53, 53, 249)     # RED
text_color = (239, 239, 239)     # WHITE
border_color = (61, 147, 8)      # GREEN

# 5. Setup Folders for Collection:

In [111]:
DATA_PATH = os.path.join("Data")

# You, Busy
actions = np.array(["Yes", "WhatAreYouDoing", "TryBeing", "ToMeet", "ThankYou", "TakeCare", "SameAsYou", "Question", "Point", "Nothing", "IHear", "HowAreYou", "Hello", "Bye", "Good", "_BLANK"])

no_sequences = 20
sequence_length = 30

In [112]:
len(actions)
actions.shape[0]

16

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 6. Collect MP Keypoints:

In [ ]:
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

def mediapipe_detections(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    results = model.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    return frame, results


def draw_landmarks(frame, results, color):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)


def extract_keypoints(results):
    hand_landmarks = np.zeros(63)
    
    if results.multi_hand_landmarks:
        hand_landmarks = np.array(
            [
                [landmark.x, landmark.y, landmark.z]
                for landmark in results.multi_hand_landmarks[0].landmark
            ]
        ).flatten()

    return hand_landmarks

In [ ]:
stream_url = "http://192.168.169.196:81/stream"
cap = cv2.VideoCapture(0)

no_frames_counter = 0
no_sequences_counter = 0
index = 15
current_action = actions[index]

while True:
    _, image = cap.read()
    
    image, results = mediapipe_detections(image, hands)
    draw_landmarks(image, results, corner_color)
    right_hand = extract_keypoints(results)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("s"):
        npy_path = os.path.join(DATA_PATH, current_action, str(no_sequences_counter), f"{no_frames_counter}.npy")
        np.save(npy_path, right_hand)
        no_frames_counter += 1
        if no_frames_counter == sequence_length:
            no_frames_counter = 0
            no_sequences_counter += 1
            if no_sequences_counter == no_sequences:
                break
    
    cv2.putText(image, f"Collecting Frames for '{current_action}'", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Video Num: {no_sequences_counter}", (15, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Frame Num: {no_frames_counter}", (15, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("Image", image)
    
    if key == ord("n"):
        cv2.imwrite(f"{current_action}-image-{no_frames_counter}-{no_sequences_counter}.jpg", image)
    
    if key == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()


In [ ]:
cap.release()
cv2.destroyAllWindows()

# 7. Preprocess Data and Create Labels and Features:

In [113]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [114]:
label_map = {label:num for num, label in enumerate(actions)}

In [115]:
label_map

{'Yes': 0,
 'WhatAreYouDoing': 1,
 'TryBeing': 2,
 'ToMeet': 3,
 'ThankYou': 4,
 'TakeCare': 5,
 'SameAsYou': 6,
 'Question': 7,
 'Point': 8,
 'Nothing': 9,
 'IHear': 10,
 'HowAreYou': 11,
 'Hello': 12,
 'Bye': 13,
 'Good': 14,
 '_BLANK': 15}

In [116]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [117]:
X = np.array(sequences)
print(X.shape) # (320, 30, 63)  ==> 63  for the inner array of hand_landmarks
               #                ==> 30  for the number of sequences
               #                ==> 320 for 16 signs x 20 (folder par sign)

(320, 30, 63)


In [118]:
y = to_categorical(labels).astype(int)

In [119]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 8. Build and Train LSTM Neural Network:

In [121]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [122]:
log_dir = os.path.join("logs")
tb_callback = TensorBoard(log_dir=log_dir) # for moritoring the NN training ==> tensorboard --logdir=.

In [124]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(30, 63)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(actions.shape[0], activation="softmax"))

In [125]:
res = [.7, .3]
actions[np.argmax(res)]

'Yes'

In [126]:
X_train.shape
y_train.shape

(304, 16)

In [127]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [128]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step - categorical_accuracy: 0.0472 - loss: 2.7690
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - categorical_accuracy: 0.1640 - loss: 2.7107
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - categorical_accuracy: 0.1866 - loss: 2.5010
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - categorical_accuracy: 0.2254 - loss: 2.3230
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - categorical_accuracy: 0.2505 - loss: 2.5477
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - categorical_accuracy: 0.2882 - loss: 2.2903
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_accuracy: 0.3111 - loss: 2.0443
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - categorical_accuracy: 0.3584 - loss: 1.7749
Epoch 9/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - categorical_accuracy: 0.4286 - loss: 1.5821
Epoch 10/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - categorical_accuracy: 0.4990 - loss: 1.3825
Epoch 11

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - categorical_accuracy: 1.0000 - loss: 0.0052
Epoch 63/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - categorical_accuracy: 1.0000 - loss: 0.0066
Epoch 64/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - categorical_accuracy: 1.0000 - loss: 0.0023
Epoch 65/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - categorical_accuracy: 1.0000 - loss: 0.0017
Epoch 66/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - categorical_accuracy: 0.9736 - loss: 0.0601
Epoch 67/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - categorical_accuracy: 0.9104 - loss: 0.2652
Epoch 68/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - categorical_accuracy: 0.9790 - loss: 0.0662
Epoch 69/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - categorical_accuracy: 0.9699 - loss: 0.1063
Epoch 70/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - categorical_accuracy: 0.9352 - loss: 0.1490
Epoch 71/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - categorical_accuracy: 0.9268 - loss: 0.3840
Epoch 72/1000


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 1.2599e-04
Epoch 123/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 2.0212e-04
Epoch 124/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 2.2096e-04
Epoch 125/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 1.6391e-04
Epoch 126/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 1.4271e-04
Epoch 127/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 1.4163e-04
Epoch 128/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 1.4510e-04
Epoch 129/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 9.8458e-05
Epoch 130/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 1.6710e-04
Epoch 131/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_a

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 4.1789e-05
Epoch 182/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 3.2634e-05
Epoch 183/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 4.2455e-05
Epoch 184/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 3.4522e-05
Epoch 185/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 1.0000 - loss: 3.3532e-05
Epoch 186/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - categorical_accuracy: 1.0000 - loss: 3.1492e-05
Epoch 187/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_accuracy: 1.0000 - loss: 3.2773e-05
Epoch 188/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_accuracy: 1.0000 - loss: 3.6657e-05
Epoch 189/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 2.7519e-05
Epoch 190/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_a

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 1.1528e-05
Epoch 241/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 1.4757e-05
Epoch 242/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 1.7836e-05
Epoch 243/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 1.0994e-05
Epoch 244/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 1.6241e-05
Epoch 245/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - categorical_accuracy: 1.0000 - loss: 1.5285e-05
Epoch 246/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_accuracy: 1.0000 - loss: 1.3369e-05
Epoch 247/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - categorical_accuracy: 1.0000 - loss: 1.1949e-05
Epoch 248/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_accuracy: 1.0000 - loss: 1.5238e-05
Epoch 249/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_a

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - categorical_accuracy: 1.0000 - loss: 1.0737e-07
Epoch 764/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - categorical_accuracy: 1.0000 - loss: 1.2271e-07
Epoch 765/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - categorical_accuracy: 1.0000 - loss: 1.2708e-07
Epoch 766/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - categorical_accuracy: 1.0000 - loss: 1.0418e-07
Epoch 767/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - categorical_accuracy: 1.0000 - loss: 1.1877e-07
Epoch 768/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - categorical_accuracy: 1.0000 - loss: 1.2596e-07
Epoch 769/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - categorical_accuracy: 1.0000 - loss: 1.2223e-07
Epoch 770/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - categorical_accuracy: 1.0000 - loss: 1.2057e-07
Epoch 771/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 1.2755e-07
Epoch 772/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - catego

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 8.9858e-08
Epoch 822/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 1.1097e-07
Epoch 823/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 6.8706e-08
Epoch 824/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - categorical_accuracy: 1.0000 - loss: 7.2349e-08
Epoch 825/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - categorical_accuracy: 1.0000 - loss: 8.8961e-08
Epoch 826/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - categorical_accuracy: 1.0000 - loss: 9.3866e-08
Epoch 827/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - categorical_accuracy: 1.0000 - loss: 8.6472e-08
Epoch 828/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - categorical_accuracy: 1.0000 - loss: 8.3671e-08
Epoch 829/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - categorical_accuracy: 1.0000 - loss: 9.0011e-08
Epoch 830/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - categor

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - categorical_accuracy: 1.0000 - loss: 6.7628e-08
Epoch 880/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_accuracy: 1.0000 - loss: 4.8962e-08
Epoch 881/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - categorical_accuracy: 1.0000 - loss: 5.7135e-08
Epoch 882/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - categorical_accuracy: 1.0000 - loss: 6.2121e-08
Epoch 883/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 1.0000 - loss: 7.5702e-08
Epoch 884/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 6.8062e-08
Epoch 885/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 6.1025e-08
Epoch 886/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 1.0000 - loss: 5.4904e-08
Epoch 887/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - categorical_accuracy: 1.0000 - loss: 3.9209e-08
Epoch 888/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_a

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - categorical_accuracy: 1.0000 - loss: 4.9658e-08
Epoch 938/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 4.4369e-08
Epoch 939/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 4.9460e-08
Epoch 940/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 3.8307e-08
Epoch 941/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 1.0000 - loss: 4.6325e-08
Epoch 942/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 3.3742e-08
Epoch 943/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 4.2283e-08
Epoch 944/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 1.0000 - loss: 3.1872e-08
Epoch 945/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 4.1366e-08
Epoch 946/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - categorical_accuracy: 1.0000 - loss: 3.3849e-08
Epoch 996/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - categorical_accuracy: 1.0000 - loss: 2.4720e-08
Epoch 997/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - categorical_accuracy: 1.0000 - loss: 2.5293e-08
Epoch 998/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - categorical_accuracy: 1.0000 - loss: 3.4794e-08
Epoch 999/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - categorical_accuracy: 1.0000 - loss: 3.3200e-08
Epoch 1000/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - categorical_accuracy: 1.0000 - loss: 2.4691e-08


In [ ]:
# model.load_weights("sign_language.keras")

# 9. Make Predictions:

In [129]:
model.summary(line_length=100)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                               ┃ Output Shape                    ┃           Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                              │ (None, 30, 64)                  │            32,768 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ lstm_10 (LSTM)                             │ (None, 30, 128)                 │            98,816 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ lstm_11 (LSTM)                             │ (None, 64)                      │            49,408 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_9 (Dense)                            │ (None, 64)                      │             4,160 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_10 (Dense)                           │ (None, 32)                      │             2,080 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_11 (Dense)                           │ (None, 16)                      │               528 │
└────────────────────────────────────────────┴─────────────────────────────────┴───────────────────┘

 Total params: 563,282 (2.15 MB)

 Trainable params: 187,760 (733.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 375,522 (1.43 MB)

In [130]:
results = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step


In [131]:
actions[np.argmax(results[0])]

'TakeCare'

In [132]:
actions[np.argmax(y_test[0])]

'TakeCare'

# 10. Save Weights:

In [133]:
model.save("sign_language__1.keras")

# 11. Evaluation using Confusion Matrix and Accuracy:

In [134]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [135]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


In [136]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [137]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[15,  0],
        [ 0,  1]],

       [[15,  0],
        [ 0,  1]],

       [[14,  0],
        [ 0,  2]],

       [[14,  0],
        [ 0,  2]],

       [[14,  0],
        [ 0,  2]],

       [[15,  0],
        [ 0,  1]],

       [[13,  0],
        [ 0,  3]],

       [[14,  0],
        [ 0,  2]],

       [[15,  0],
        [ 0,  1]],

       [[15,  0],
        [ 0,  1]]], dtype=int64)

In [138]:
accuracy_score(ytrue, yhat)

1.0

In [139]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(results, actions, input_frame, colors):
    output_frame = input_frame.copy()
    output_frame = cv2.resize(output_frame, (1280, 960))
    
    for num, prob in enumerate(results):
        cv2.rectangle(output_frame, (0, num*40), (int(prob*100), 40+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num], (0, 32+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# 12. Test Real Time:

In [147]:
sequence = []
sentence = []
# predictions = []
threshold = 0.5

# model.load_weights("sign_language__1.keras")

stream_url = "http://192.168.169.196:81/stream"
cap = cv2.VideoCapture(stream_url)

while True:
    _, frame = cap.read()

    frame, results = mediapipe_detections(frame, hands)
    draw_landmarks(frame, results, corner_color)

    right_hand = extract_keypoints(results)
    sequence.append(right_hand)
    sequence = sequence[-30:]
    
    if len(sequence) == 30:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        # predictions.append(np.argmax(res))
    
        # if np.unique(predictions[-10:])[0] == np.argmax(res):
        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
            
    frame = prob_viz(res, actions, frame, colors)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

In [ ]:
cap.release()
cv2.destroyAllWindows()